QUICK FEATURE PREP

In [1]:
import pandas as pd
import numpy as np


mad_df = pd.read_csv("mad_center_data.csv", delimiter=";", parse_dates=["FECHA"], index_col="FECHA")
mad_df.shape #(79554, 35)
df = mad_df[mad_df["FAMILIA"] == "TABLERO"]
df = df[df["CANT."] <= 100] #outliers
df= df[df["Espesor / Medida"].isin(["15MM", "18MM"])]

df.shape #(23662, 35)
#df.columns
df = df[["SUCURSAL", "DESCRIPCIÓN", "CANT.", "PRECIO", "TOTALF"]]

display(df.sample(5))

C:\Users\usuario\AppData\Local\Temp\ipykernel_6592\4199717582.py:5: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  mad_df = pd.read_csv("mad_center_data.csv", delimiter=";", parse_dates=["FECHA"], index_col="FECHA")


,SUCURSAL,DESCRIPCIÓN,CANT.,PRECIO,TOTALF
FECHA,,,,,
2025-07-18,Alm Suc 6 Cota Cota,MDP PELIK MEL 2C CARAMELO 15mm,9.0,854.0,7686.0
2024-10-29,Alm Suc2 Juan Pablo II,MDP EUX MEL 2C FREIJO CAF SB NG 15MM,2.0,359.0,718.0
2025-03-20,Alm Suc2 Juan Pablo II,MDP PELIK MEL 2C ONIX 15MM,12.0,640.0,7680.0
2024-11-25,Alm Suc 6 Cota Cota,MDP PELIK MEL 2C NIEBLA 18MM,1.0,667.0,667.0
2025-07-02,Alm Suc 6 Cota Cota,MDP PELIK MEL 2C SYNC ALAMO BARDOLINO 15MM,1.0,944.0,944.0


PRODUCT DEMAND FORECAST MODEL

In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

df = df.copy()

df["MONTH"] = df.index.to_period("M").astype(str)

monthly = (df.groupby(["DESCRIPCIÓN", "MONTH"])["CANT."].sum().reset_index())

pipe = Pipeline([
    ("enc", ColumnTransformer(
        transformers=[
            ("desc", OneHotEncoder(handle_unknown='ignore'), ["DESCRIPCIÓN"])
        ],
        remainder='drop'
    )),
    ("model", RandomForestRegressor(n_estimators=200, random_state=42))
])

X = monthly[["DESCRIPCIÓN"]]
y = monthly["CANT."]
pipe.fit(X, y)

def predecir_demanda_mensual(descripcion_producto):
    pred = pipe.predict(pd.DataFrame({"DESCRIPCIÓN": [descripcion_producto]}))[0]
    return round(pred, 2)


MODEL EVALUATION

In [3]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

r2 = r2_score(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)

mape = np.mean(np.abs((y_test - y_pred) / np.maximum(y_test, 1))) * 100

rmsle = np.sqrt(mean_squared_error(
    np.log1p(y_test), np.log1p(y_pred)
))

print("🔹 R²:", round(r2, 4))
print("🔹 RMSE:", round(rmse, 4))
print("🔹 MAE:", round(mae, 4))
print("🔹 MAPE (%):", round(mape, 2))
print("🔹 RMSLE:", round(rmsle, 4))


🔹 R²: 0.855
🔹 RMSE: 26.0603
🔹 MAE: 13.2853
🔹 MAPE (%): 206.29
🔹 RMSLE: 0.9215


c:\Users\usuario\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


MODEL READY TO EXECUTE

In [4]:
producto = "MDP PELIK MEL 2C CARAMELO 18mm"
prediccion = predecir_demanda_mensual(producto)
print(f"Demanda mensual estimada para '{producto}': {prediccion} unidades")

Demanda mensual estimada para 'MDP PELIK MEL 2C CARAMELO 18mm': 82.94 unidades
